### VLLM serve command:

### Codestral-22B-v0.1
``` sh
docker run --rm --gpus all --privileged   --ipc=host  --shm-size=2g --network host -v ~/.cache/huggingface:/root/.cache/huggingface
  -v /home/odedh/SML-For-Debug/models/:/models   vllm/vllm-openai:latest   --model /models/Codestral-22B-v0.1   --use-tqdm-on-load   
--enable-chunked-prefill   --tensor-parallel-size 2

```

### Qwen3-0.6B
``` sh
docker run --rm --gpus all  --privileged   --ipc=host  --shm-size=2g --network host -v ~/.cache/huggingface:/root/.cache/huggingface -v /home/odedh/SML-For-Debug/models/:/models   -e CUDA_VISIBLE_DEVICES=2 vllm/vllm-openai:latest   --model /models/Qwen3-0.6B   --use-tqdm-on-load   --enable-chunked-prefill    --port 8001
```


### Llama-3.2-1B-Instruct
``` sh
docker run --rm --gpus all  --privileged   --ipc=host  --shm-size=2g --network host -v ~/.cache/huggingface:/root/.cache/huggingface -v /home/odedh/SML-For-Debug/models/:/models   -e CUDA_VISIBLE_DEVICES=2 vllm/vllm-openai:latest   --model /models/Llama-3.2-1B-Instruct   --use-tqdm-on-load   --enable-chunked-prefill    --port 8001

```

### GLM-4.5-Air
``` sh
docker run --rm --gpus all --privileged   --ipc=host  --shm-size=16g --network host -v ~/.cache/huggingface:/root/.cache/hggingface  -v /mnt/hf_models/:/models   vllm/vllm-openai:latest   --model /models/GLM-4.5-Air   --use-tqdm-on-load   --enable-chunked-prefill   --tensor-parallel-size=8  --port 8001
```

In [2]:
# !pip install openai

Running a simpel example for the Qwen3-0.6B

In [3]:
from openai import OpenAI

client = OpenAI(
    base_url="http://localhost:8001/v1",
    api_key="-",
)
# print the model name
print(client.models.list().data[0].id)

model = client.models.list().data[0].id

code_for_debugging = """
def divide_numbers(a, b):
    return a / b
"""
content = f"Classify this code and ans if there is a bug in it: {code_for_debugging}"


completion = client.chat.completions.create(
    model=model,
    temperature=0.1,
    messages=[
        {"role": "user", "content": content}
    ],
    extra_body={"guided_choice": ["bug", "no-bug"]},
    
)
print(completion.choices[0].message.content)

/models/Qwen3-0.6B
bug


Codestral-22B-v0.1

In [ ]:
from openai import OpenAI

client = OpenAI(
    base_url="http://localhost:8001/v1",
    api_key="-",
    
)
model = client.models.list().data[0].id
print(model)

code_for_debugging = """
def _make_sqlite_account_info(self, env=None, last_upgrade_to_run=None):\n\t\twith mock.patch('os.environ', env or {'HOME': self.home}):\n\t\t\treturn SqliteAccountInfo(\n\t\t\t\tfile_name=self.db_path if not env else None,\n\t\t\t\tlast_upgrade_to_run=last_upgrade_to_run,\n\t\t\t)
"""
content = f"Fix this code to have no bugs: {code_for_debugging}\n\n just return the fixed version of the function, no explanation needed"


completion = client.chat.completions.create(
    model=model,
    temperature=0.1,
    messages=[
        {"role": "user", "content": content}
    ],
    # extra_body={"guided_choice": ["bug", "no-bug"]},
    
)
print(completion.choices[0].message.content)

/models/Qwen3-0.6B
<think>
Okay, let's see. The user wants me to fix the code so that there are no bugs. The original code is:

def divide_numbers(a, b):
    return a / b

And they want just the fixed version of the function, no explanation needed.

First, I need to check if there are any syntax errors. The function is correctly defined with parameters a and b, and the return statement is a simple division. The code looks straightforward.

Wait, maybe there's a problem with the return statement? Like, does it handle division by zero? But the user didn't mention that. They just want the function to return a / b. So perhaps the function is correct as is.

Another thing to check: the function name is 'divide_numbers', which is a common name. The parameters are a and b, which are correct. The return value is a division. So no issues there.

Is there any possible bug in the code? For example, if a or b is not a number, but the function is supposed to handle that. However, the problem statem

In [2]:
from pydantic import BaseModel, Field
from enum import Enum
from typing import Optional, List

class BugStatus(str, Enum):
    no_bug = "no_bug"
    has_bug = "has_bug"
    # uncertain = "uncertain"

class Fixability(str, Enum):
    fixable = "fixable"
    not_fixable = "not_fixable"
    unknown = "unknown"

class ClassificationResult(BaseModel):
    has_bug: BugStatus
    bug_type: Optional[str] = Field(default=None, description="Type of bug if present")
    suggested_fix: Optional[str] = Field(default=None, description="Optional fix attempt if model detects a bug")
    confidence_score: float = Field(default=0, ge=0.0, le=1.0)
    fixability: Optional[Fixability] = None
    tested_functions: Optional[List[str]] = None
    rationale: Optional[str] = Field(default=None, description="Short reasoning for classification")

json_schema = ClassificationResult.model_json_schema()


code_snippet = """
def get_api_response_ext(self, http_resp, url='/images', headers={},\n\t\t\t\t\t\t\t body=None, method=None, api=None,\n\t\t\t\t\t\t\t content_type=None):\n\t\tif api is None:\n\t\t\tapi = self.api\n\t\treq = webob.Request.blank(url)\n\t\tfor k, v in headers.iteritems():\n\t\t\treq.headers[k] = v\n\t\tif method:\n\t\t\treq.method = method\n\t\tif body:\n\t\t\treq.body = body\n\t\tif content_type == 'json':\n\t\t\treq.content_type = 'application/json'\n\t\telif content_type == 'octet':\n\t\t\treq.content_type = 'application/octet-stream'\n\t\tres = req.get_response(api)\n\t\tself.assertEqual(res.status_int, http_resp)\n\t\treturn res
"""

description = """
Modify the get_api_response_ext method to introduce a Missing If Construct Plus Statements (MIFS) fault. The function should fail due to removing the check if headers is None:, causing potential TypeError exceptions when headers is None and iteriting over it.
"""

completion = client.chat.completions.create(
    model=model,
    temperature=0.15, # Lower temperature for more deterministic output
    messages=[
        {
            "role": "system",
            "content": (
                "You are given a Python code snippet. Your job is to classify whether it contains a bug. "
                "If it does, describe the bug and attempt a fix. If not, state that it is correct. "
                "Return your response as a structured JSON matching the provided schema. "
                "Do not explain or justify in natural language outside the JSON. "
                "Do not assume the answer is known. Use your reasoning to classify the code."
            )
        },
        {
            "role": "user",
            "content": f"The code snippet: {code_snippet}\nShort description: {description}"
        }
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "code-diagnostic",
            "schema": json_schema
        },
    },
)

print(completion.choices[0].message.content)


{"has_bug": "has_bug", "bug_type": "Missing If Construct Plus Statements (MIFS)", "fixability": "not_fixable", "tested_functions": ["get_api_response_ext"] , "rationale": "The function is missing an if statement to check if headers is None, which could lead to a TypeError when headers is None. This is a MIFS fault because the code does not include the if condition, and the fix is not possible to implement."}
